<a href="https://colab.research.google.com/github/Zain-Pirzada/Diabetic-Reinopathy-Kaggle-Competition/blob/main/Transfer_Learning_VGG_16_%26_VGG_19_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Transfer Learning VGG 16 and VGG 19 using Keras**

**Import data from gogle drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import Keras**

In [ ]:
!pip install -q keras   

In [2]:
import keras

In [ ]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 588.3 MB 22 kB/s 
     |████████████████████████████████| 439 kB 74.7 MB/s 
     |████████████████████████████████| 1.7 MB 59.0 MB/s 
     |████████████████████████████████| 6.0 MB 47.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully u

In [ ]:
!nvidia-smi

Thu Dec 22 12:40:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Transfer Learning using VGG 16 & VGG 19 Using Keras**

In [3]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [4]:
import tensorflow as tf
print(tf.__version__)


2.9.2


In [5]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [6]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/DATASETS__resize_split/val'
valid_path = '/content/drive/MyDrive/DATASETS__resize_split/test'

In [7]:
# Import the VGG16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg19 = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

80134624/80134624 [==============================] - 1s 0us/step


In [10]:
# don't train existing weights
for layer in vgg19.layers:
    layer.trainable = True


In [11]:
# useful for getting number of output classes
folders = glob('/content/drive/MyDrive/DATASETS__resize_split/val*')

In [12]:
folders

['/content/drive/MyDrive/DATASETS__resize_split/val']

In [13]:
# our layers - you can add more if you want
x = Flatten()(vgg19.output)

In [14]:
len(folders)

1

In [15]:
prediction = Dense(5, activation='softmax')(x)

# create a model object
model = Model(inputs=vgg19.input, outputs=prediction)

In [16]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [17]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [18]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [19]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/DATASETS__resize_split/val',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 7033 images belonging to 5 classes.


In [20]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/DATASETS__resize_split/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 7030 images belonging to 5 classes.


In [21]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-21-2d02736eff38>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/20
220/220 [==============================] - 4353s 20s/step - loss: 2.0331 - accuracy: 0.7281 - val_loss: 0.8726 - val_accuracy: 0.7343
Epoch 2/20
220/220 [==============================] - 154s 700ms/step - loss: 0.8757 - accuracy: 0.7354 - val_loss: 0.8831 - val_accuracy: 0.7343
Epoch 3/20
220/220 [==============================] - 154s 701ms/step - loss: 0.8786 - accuracy: 0.7354 - val_loss: 0.8697 - val_accuracy: 0.7343
Epoch 4/20
220/220 [==============================] - 152s 691ms/step - loss: 0.8739 - accuracy: 0.7354 - val_loss: 0.8730 - val_accuracy: 0.7343
Epoch 5/20
220/220 [==============================] - 153s 694ms/step - loss: 0.8812 - accuracy: 0.7354 - val_loss: 0.8767 - val_accuracy: 0.7343
Epoch 6/20
220/220 [==============================] - 154s 701ms/step - loss: 1.0305 - accuracy: 0.7321 - val_loss: 0.8753 - val_accuracy: 0.7343
Epoch 7/20
220/220 [==============================] - 152s 692ms/step - loss: 0.8743 - accuracy: 0.7354 - val_loss: 0.8725 - 

In [23]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')


NameError: ignored